# 7 - Audio a Audio 


<br>
<br>

<img src="https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/audio2audio.webp" style="width:400px;"/>

<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1---Modelo-Audio-a-Audio" data-toc-modified-id="1---Modelo-Audio-a-Audio-1">1 - Modelo Audio a Audio</a></span></li><li><span><a href="#2---Modelo-separador-de-fuentes" data-toc-modified-id="2---Modelo-separador-de-fuentes-2">2 - Modelo separador de fuentes</a></span></li><li><span><a href="#3---Modelo-conversión-de-voz" data-toc-modified-id="3---Modelo-conversión-de-voz-3">3 - Modelo conversión de voz</a></span></li></ul></div>

## 1 - Modelo Audio a Audio

Los modelos audio-to-audio son sistemas de inteligencia artificial diseñados para transformar una señal de audio en otra señal de audio, aplicando diversas técnicas de procesamiento y generación de audio. Estos modelos tienen una amplia gama de aplicaciones en áreas como la mejora del habla, la síntesis de voz, la separación de fuentes musicales y la conversión de estilo de audio. 


**Aplicaciones principales**


1. **Mejora del habla**:

    + Reducción de ruido: Eliminar el ruido de fondo de grabaciones de audio, mejorando la claridad del habla.
    + Aclaración del audio: Mejorar la calidad del audio, haciendo que las grabaciones sean más comprensibles.


2. **Síntesis de voz**:

    + Conversión de voz: Transformar la voz de un hablante en la voz de otro hablante manteniendo el contenido semántico del discurso.
    + Alteración del estilo de voz: Cambiar el tono, la velocidad o la emoción de una grabación de voz.


3. **Separación de fuentes musicales**:

    + Separación de instrumentos: Extraer diferentes instrumentos de una mezcla de música, permitiendo el análisis o procesamiento individual de cada instrumento.
    + Separación de voces y música: Aislar la voz del cantante de la música de fondo en una canción.


4. **Conversión de estilo de audio**:

    + Transferencia de estilo: Transformar un audio de un estilo particular, por ejemplo clásico, a otro estilo, por ejemplo jazz.
    + Efectos de audio: Aplicar efectos de audio específicos, como reverberación, eco o distorsión a una grabación de audio.

<br>

**Componentes Principales**


1. **Autoencoders**: Utilizados para comprimir y descomprimir las señales de audio, capturando las características esenciales del audio en un espacio latente más manejable. Esto facilita la transformación y mejora del audio.


2. **Redes neuronales recurrentes (RNN) y Transformers**: Modelos avanzados de aprendizaje profundo que manejan secuencias temporales, como el audio. Los transformers, en particular, son eficaces para captar dependencias a largo plazo en las señales de audio.


3. **Generative Adversarial Networks (GANs)**: Utilizadas para generar señales de audio realistas a partir de representaciones latentes, aprendiendo a imitar las características del audio humano.


4. **Procesamiento digital de señales (DSP)**: Técnicas tradicionales de procesamiento de audio que a menudo se combinan con métodos de aprendizaje profundo para mejorar la calidad y efectividad de las transformaciones de audio.

<br>

**Ejemplo de Modelos Audio-to-Audio**

+ Voice Conversion Models: Modelos como StarGAN-VC y CycleGAN-VC son utilizados para la conversión de voz, transformando una grabación de voz en otra manteniendo el contenido semántico.


+ Speech Enhancement Models: Modelos como Deep Speech Enhancement (DSE) se enfocan en mejorar la calidad del habla en ambientes ruidosos, utilizando técnicas de aprendizaje profundo para reducir el ruido de fondo.


+ Music Source Separation: Herramientas como Open-Unmix y Spleeter permiten la separación de diferentes componentes en una mezcla musical, facilitando tareas como la creación de remixes o la mejora de grabaciones.


## 2 - Modelo separador de fuentes

[Sepformer](https://huggingface.co/speechbrain/sepformer-wsj03mix) es un modelo creado por [SpeechBrain](https://speechbrain.github.io/) capaz de separar las fuentes de audio. Vamos a probarlo con un audio que tiene tres voces distintas y le vamos a pedir que las separe en tres audios distintos. Tiene un peso de unos 200Mb. Para usar este modelo debemos realizar la siguiente instalación:

```bash
pip install speechbrain
```

In [1]:
# para quitar warnings 

from transformers import logging
logging.set_verbosity_error()

import warnings
warnings.filterwarnings('ignore')

In [2]:
# librerías

import torchaudio
from speechbrain.inference.separation import SepformerSeparation 
import librosa
from IPython.display import Audio

In [3]:
# audio original

ruta = 'audio_cache/test_mixture_3spks.wav'

data, freq = librosa.load(ruta)

Audio(data=data, rate=freq)

In [4]:
# modelo separador

modelo = SepformerSeparation.from_hparams(source='speechbrain/sepformer-wsj03mix')

In [5]:
# separando fuentes

fuentes = modelo.separate_file(path='speechbrain/sepformer-wsj03mix/test_mixture_3spks.wav')

In [6]:
# 1ª fuente

Audio(data=fuentes[:, :, 0].detach().cpu(), rate=8000)

In [7]:
# 2ª fuente

Audio(data=fuentes[:, :, 1].detach().cpu(), rate=8000)

In [8]:
# 3ª fuente

Audio(data=fuentes[:, :, 2].detach().cpu(), rate=8000)

## 3 - Modelo conversión de voz

El modelo [speecht5_vc](https://huggingface.co/microsoft/speecht5_vc) de Microsoft es capaz de cambiar el estilo de una voz, por ejemplo cambiando el género del hablante. Vamos a cambiar de masculino a femenino un audio de voz. El modelo tiene un peso aproximado de 700Mb.

In [9]:
# librerías

import torch
import numpy as np
from datasets import load_dataset
from transformers import SpeechT5Processor, SpeechT5ForSpeechToSpeech, SpeechT5HifiGan

In [10]:
# carga de audio original con voz masculina desde un dataset

dataset = load_dataset('hf-internal-testing/librispeech_asr_demo', 'clean', split='validation')


data = dataset[0]['audio']['array']

freq = dataset.features['audio'].sampling_rate


Audio(data=data, rate=freq)

In [11]:
# carga del modelo, el procesador y el vocoder


ruta = 'microsoft/speecht5_vc'


procesador = SpeechT5Processor.from_pretrained(ruta)

modelo = SpeechT5ForSpeechToSpeech.from_pretrained(ruta)

vocoder = SpeechT5HifiGan.from_pretrained('microsoft/speecht5_hifigan')  

In [12]:
# creación del tensor

tensor = procesador(audio=data, sampling_rate=freq, return_tensors='pt')

tensor

{'input_values': tensor([[0.0024, 0.0021, 0.0020,  ..., 0.0004, 0.0006, 0.0010]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}

In [13]:
# embedding de audio femenino

embedding_data = load_dataset('Matthijs/cmu-arctic-xvectors', split='validation')

embedding = torch.tensor(embedding_data[7306]['xvector']).unsqueeze(0)

In [14]:
# generación de voz femenina

resultado = modelo.generate_speech(tensor['input_values'], embedding, vocoder=vocoder)

In [15]:
# escuchamos nuevo audio

Audio(data=resultado.numpy(), rate=16000)